In [1]:
import pandas as pd
from binomial_european import binomial_call

In [2]:
options_data_df = pd.DataFrame(
    [['NVDA',103,10,0.2],
     ['NVDA',110,4,0.3]],
    columns=['ticker', 'strike_price', 'ask_price', 'volatility']
)
options_data_df

,ticker,strike_price,ask_price,volatility
0,NVDA,103,10,0.2
1,NVDA,110,4,0.3


In [3]:
stock_data_df = pd.DataFrame(
    [['NVDA',100,0.1,0.5,3],
     ['AMDS',100,0.1,0.5,3]], #filters by ticker
    columns=['ticker', 'stock_price', 'interest_rate', 'time_to_expiry', 'steps']
)
stock_data_df

,ticker,stock_price,interest_rate,time_to_expiry,steps
0,NVDA,100,0.1,0.5,3
1,AMDS,100,0.1,0.5,3


In [4]:
combined_df = pd.merge(options_data_df, stock_data_df, how='inner', on='ticker') #filters by ticker
combined_df

,ticker,strike_price,ask_price,volatility,stock_price,interest_rate,time_to_expiry,steps
0,NVDA,103,10,0.2,100,0.1,0.5,3
1,NVDA,110,4,0.3,100,0.1,0.5,3


In [5]:
combined_df.loc[:,'fair_price'] = combined_df.apply(lambda x: binomial_call(
    S0=x['stock_price'],
    K=x['strike_price'],
    T=x['time_to_expiry'],
    r=x['interest_rate'],
    N=x['steps'],
    sig=x['volatility']
    ), axis=1)
combined_df

,ticker,strike_price,ask_price,volatility,stock_price,interest_rate,time_to_expiry,steps,fair_price
0,NVDA,103,10,0.2,100,0.1,0.5,3,6.878098
1,NVDA,110,4,0.3,100,0.1,0.5,3,6.247340


In [6]:
def valuation(ask_price,fair_price):
    if ask_price > fair_price:
        return 'overvalued'
    elif ask_price < fair_price:
        return 'undervalued'
    else:
        return 'fair value'

In [7]:
combined_df.loc[:,'valuation'] = combined_df.apply(lambda x: valuation(x['ask_price'], x['fair_price']), axis=1)
combined_df

,ticker,strike_price,ask_price,volatility,stock_price,interest_rate,time_to_expiry,steps,fair_price,valuation
0,NVDA,103,10,0.2,100,0.1,0.5,3,6.878098,overvalued
1,NVDA,110,4,0.3,100,0.1,0.5,3,6.247340,undervalued
